In [1]:
import pandas as pd 
import plotly.graph_objects as go
import sys 
from atlantic_temps import TempPredictionEnv

In [2]:
fp = '../datasets/atlantic_sea_temp.jsonl'
df = pd.read_json(fp, lines=True)
df

,year,day,temp
0,2023,0,20.78
1,2022,0,20.85
2,2021,0,20.66
3,2020,0,20.60
4,2019,0,20.30
...,...,...,...
15733,1985,365,NaN
15734,1984,365,19.84
15735,1983,365,NaN
15736,1982,365,NaN


In [3]:
# from stable_baselines3.common.env_checker import check_env
# env = TempPredictionEnv(fp)

# # Check the environment
# initial_observation = env.reset()
# print(f'Initial observation: {initial_observation}')

# next_observation, reward, terminated, truncated, _ = env.step([1.0])  # taking a random action
# print(f'Next observation: {next_observation}, Reward: {reward}, Done: {terminated}')

In [4]:
from stable_baselines3 import A2C, PPO
import os 

env = TempPredictionEnv(fp)

Removed 375 NaN values from the dataset.


In [5]:
TIMESTEPS = 50000

name = 'atlantic_ss'
models_dir = os.path.join('models', name)
logdir = "../logs"

model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=logdir, device='mps')
model.learn(total_timesteps=TIMESTEPS, tb_log_name=name)

Using mps device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ../logs/atlantic_ss_2
------------------------------------
| time/                 |          |
|    fps                | 112      |
|    iterations         | 100      |
|    time_elapsed       | 4        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.41    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -99.7    |
|    std                | 0.987    |
|    value_loss         | 9.79e+03 |
------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 133       |
|    iterations         | 200       |
|    time_elapsed       | 7         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -1.41     |
|    explained

In [6]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

eval_env = Monitor(TempPredictionEnv(fp), logdir)
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=1, deterministic=False)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}") 

Removed 375 NaN values from the dataset.
mean_reward=-385779.29 +/- 0.0


In [7]:
obs, info = env.reset()
total_rewards = 0
while True:
    action, _states = model.predict(obs)
    obs, rewards, term, trunc, info = env.step(action)
    done = term or trunc
    total_rewards += rewards
    if done:
        print("info", info)
        break
env.close()

info {}


In [8]:
total_rewards

-385394.56917809066